<a href="https://colab.research.google.com/github/hjn14133/Machine-Learning/blob/main/LightGBM_yield_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Initial Setup
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
from sklearn import metrics

In [2]:
# Importing the dataset
df = pd.read_csv("FT_join_JH_cleaned.csv")

#Select columns
X = df[['DISTANCE', 'Heading','Elevation', 'AppliedRat']]
y = df['VRYIELDBAL']

In [3]:
# split the dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, 
                                                    random_state = 0)

In [4]:
import time
start = time.time()
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
lgb=lgb.LGBMRegressor(objective = 'regression')
parameters = {'num_leaves':[20,30,40,60,80], 
              'min_child_samples':[5, 10, 15, 20, 25],
              'learning_rate':[0.01, 0.05,0.1,0.2],
              'reg_alpha':[0,0.01,0.03]}

clf=RandomizedSearchCV(lgb,parameters,scoring='r2',n_iter=100)
clf.fit(X=X_train, y=y_train)
print(clf.best_params_)
predicted=clf.predict(X_test)
print('MAE of Regression of the result is:')
print(mean_absolute_error(y_test, predicted))
end = time.time()
print('Execution time is:')
print(end - start)

{'reg_alpha': 0.03, 'num_leaves': 60, 'min_child_samples': 10, 'learning_rate': 0.1}
MAE of Regression of the result is:
0.3769595089834778
Execution time is:
94.09699773788452


In [5]:
print('Training Coefficient of R^2 : %.3f'%clf.score(X_train, y_train))
print('Test Coefficient of R^2 : %.3f'%clf.score(X_test, y_test))

Training Coefficient of R^2 : 0.828
Test Coefficient of R^2 : 0.659


In [7]:
# Calculate the absolute errors
errors = abs(predicted - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), '.')


# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Mean Absolute Error: 0.38 .
Accuracy: 68.41 %.


In [8]:
# Print result of MAE
print(metrics.mean_absolute_error(y_test, predicted))
# Print result of MSE
print(metrics.mean_squared_error(y_test, predicted))

# Print result of RMSE
print(np.sqrt(metrics.mean_squared_error(y_test, predicted)))

0.3769595089834778
0.28137528970774883
0.5304481970067848
